## An In-depth Exploration of US Job Market by analyzing LinkedIn Job Postings using Natural Language Processing Techniques

## Data ingestion and preprocesing
We firstly need to read the webscraped dataset and explore its contents before we begin the analysis.

In [4]:
# importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import re

Matplotlib is building the font cache; this may take a moment.


In [6]:
# read the dataset
data = pd.read_csv('/Users/juliabarsow/Desktop/thesis/project_code/postings.csv')

In [7]:
# exploring the first two rows of the dataset
pd.set_option('display.max_columns', None)
data.head(2)

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,min_salary,formatted_work_type,applies,original_listed_time,remote_allowed,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,17.0,Full-time,2.0,1.713398e+12,NaN,https://www.linkedin.com/jobs/view/921716/?trk...,NaN,ComplexOnsiteApply,1.715990e+12,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,30.0,Full-time,NaN,1.712858e+12,NaN,https://www.linkedin.com/jobs/view/1829192/?tr...,NaN,ComplexOnsiteApply,1.715450e+12,NaN,NaN,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0


In [8]:
data.shape

(123849, 31)

In [9]:
#check how many descriptions qare missing as this is the most important column
print("Missing rows of description: ",data['description'].isnull().sum())

Missing rows of description:  7


In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
job_id,123849.0,3.896402e+09,8.404355e+07,9.217160e+05,3.894587e+09,3.901998e+09,3.904707e+09,3.906267e+09
max_salary,29793.0,9.193942e+04,7.011101e+05,1.000000e+00,4.828000e+01,8.000000e+04,1.400000e+05,1.200000e+08
company_id,122132.0,1.220401e+07,2.554143e+07,1.009000e+03,1.435200e+04,2.269650e+05,8.047188e+06,1.034730e+08
views,122160.0,1.461825e+01,8.590360e+01,1.000000e+00,3.000000e+00,4.000000e+00,8.000000e+00,9.975000e+03
med_salary,6280.0,2.201562e+04,5.225587e+04,0.000000e+00,1.894000e+01,2.550000e+01,2.510500e+03,7.500000e+05
min_salary,29793.0,6.491085e+04,4.959738e+05,1.000000e+00,3.700000e+01,6.000000e+04,1.000000e+05,8.500000e+07
applies,23320.0,1.059198e+01,2.904739e+01,1.000000e+00,1.000000e+00,3.000000e+00,8.000000e+00,9.670000e+02
original_listed_time,123849.0,1.713152e+12,4.848209e+08,1.701811e+12,1.712863e+12,1.713395e+12,1.713478e+12,1.713573e+12
remote_allowed,15246.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
expiry,123849.0,1.716213e+12,2.321394e+09,1.712903e+12,1.715481e+12,1.716042e+12,1.716088e+12,1.729125e+12


In [7]:
#checking for missing data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [8]:
data.dtypes

job_id                          int64
company_name                   object
title                          object
description                    object
max_salary                    float64
pay_period                     object
location                       object
company_id                    float64
views                         float64
med_salary                    float64
min_salary                    float64
formatted_work_type            object
applies                       float64
original_listed_time          float64
remote_allowed                float64
job_posting_url                object
application_url                object
application_type               object
expiry                        float64
closed_time                   float64
formatted_experience_level     object
skills_desc                    object
listed_time                   float64
posting_domain                 object
sponsored                       int64
work_type                      object
currency    

In [9]:
# ✅ Calculate missing values, available count, and percentage
missing_values = data.isnull().sum().to_frame(name='missing_count')
missing_values['available_count'] = len(data) - missing_values['missing_count']
missing_values['missing_percentage'] = (missing_values['missing_count'] / len(data)) * 100  # Keep as float

# ✅ Reorder columns for better readability (available_count first)
missing_values = missing_values[['available_count', 'missing_count', 'missing_percentage']]

# ✅ Sort by missing percentage (ascending for better features on top)
missing_values = missing_values.sort_values(by='missing_percentage', ascending=True)

# ✅ Reset index and rename it to "column_name"
missing_values = missing_values.reset_index().rename(columns={'index': 'column_name'})

# ✅ Apply clear and meaningful color formatting
styled_missing_values = (
    missing_values.style
    .background_gradient(subset=['available_count'], cmap='Greens')  # More available → Green (good)
    .background_gradient(subset=['missing_count'], cmap='Oranges_r')  # More missing → Darker Orange (bad)
    .background_gradient(subset=['missing_percentage'], cmap='Reds_r')  # Higher missing % → Darker Red (bad)
    .format({'missing_percentage': "{:.2f}%"})  # Format as percentage AFTER styling
)

# ✅ Display dataset size
print(f"The dataset size: {data.shape[0]} rows")

# ✅ Display missing values table with improved color usage
display(styled_missing_values)

The dataset size: 123849 rows


,column_name,available_count,missing_count,missing_percentage
0,job_id,123849,0,0.00%
1,work_type,123849,0,0.00%
2,sponsored,123849,0,0.00%
3,listed_time,123849,0,0.00%
4,expiry,123849,0,0.00%
5,application_type,123849,0,0.00%
6,original_listed_time,123849,0,0.00%
7,formatted_work_type,123849,0,0.00%
8,job_posting_url,123849,0,0.00%
9,title,123849,0,0.00%


As we can see, there's a significant amount of missing data, however we will drop columns for every usecase we have

In [10]:
data['fips'].value_counts()

fips
36061.0    2765
6037.0     2396
17031.0    2360
48113.0    2263
48201.0    2146
           ... 
2220.0        1
29053.0       1
38049.0       1
47055.0       1
21105.0       1
Name: count, Length: 2270, dtype: int64

preprocessing to perform:
1. lowercasing
2. noise removal -> removing punctuations, emoticon, hashtags, accent marks or diacritics, extra white spaces, special characters, digits (could be useful for sentiment analysis though!!!!)
3. stop word removal -> removing stop words, sparse terms, and particular words. You can use already existing stop words lists or you can create a custom one for your use case.
4. tokenization -> breaking it down into smaller, minimal, meaningful units to work with. It enables to analyze each element in context of the other elements
5. lemmatization/stemming -> reduce the words to their root forms, reducing the bias introduced by the inflection, Lemmatization transforms words to the actual root. You have to know the POS of the word to get the correct lemma. The root form, in the stemming case, is a truncated one: stemming is a process that chops off the ends of words.
6. token enrichment -> POS tagging -> gives a mark to words based on the part-of speech they are, such as nouns, verbs and adjectives

## Skill Extraction and Clustering

### Objective: Extract required skills from job descriptions and cluster them to identify common skill sets across industries.

NLP Techniques: Named Entity Recognition (NER), Topic Modeling, or Clustering algorithms.

Research Questions:
○ What are the most in-demand skills across different sectors?
○ How do skill requirements differ by salary range or job title?
○ What are the salary ranges in different sectors and job positions?


## Preprocessing the dataset for skill extraction

In [35]:
# Load the small English spaCy model.
# This model includes lemmatization, stop words, and tokenization capabilities.
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading 'en_core_web_sm' model...")
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

In [38]:
def preprocess_text(text: str) -> str:
    """
    Performs a series of text preprocessing steps:
    1. Removes special characters and numbers.
    2. Converts text to lowercase.
    3. Tokenizes the text.
    4. Removes stop words.
    5. Lemmatizes the tokens.

    Args:
        text: The raw text string to be preprocessed.

    Returns:
        The preprocessed and cleaned text string.
    """
    # Check if the input is a valid string. If not, return an empty string.
    if not isinstance(text, str):
        return ""

    # 1. Remove special characters, punctuation, and numbers.
    # We'll keep spaces and letters.
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)

    # Convert to a spaCy Doc object for efficient processing.
    doc = nlp(text)

    # 2. Convert to lowercase, remove leading/trailing whitespace, and perform
    #    stop word removal and lemmatization.
    # A list comprehension is used for efficiency.
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop and not token.is_punct and token.is_alpha
    ]

    # Join the processed tokens back into a single string.
    return " ".join(tokens)


In [37]:
def preprocess_job_descriptions(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Applies the text preprocessing function to a specified column in a DataFrame.

    Args:
        df: The pandas DataFrame containing the raw data.
        column_name: The name of the column with the job description text.

    Returns:
        A new DataFrame with a 'preprocessed_text' column.
    """
    # Ensure the specified column exists in the DataFrame.
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in DataFrame.")
        return df

    # Apply the preprocess_text function to each entry in the column.
    print("Preprocessing job descriptions...")
    df['preprocessed_text'] = df[column_name].apply(preprocess_text)
    print("Preprocessing complete!")

    return df

In [40]:
# making a copy of the dataset for further processing
# Random Sampling
# The dataframe contains 123,849 rows, embedding all rows will lead to excessive computational cost for this demo. We will select 1000 rows for job postings.
df = data.sample(1000, random_state=42).copy()
# keeping only the relevant columns for skill extraction
df = df[['job_id', 'description']]

print("Original DataFrame:")
print(df)
print("\n" + "-"*30 + "\n")

# Preprocess the 'description' column.
preprocessed_df = preprocess_job_descriptions(df, 'description')

# Print the resulting DataFrame to see the preprocessed text.
print("Preprocessed DataFrame:")
print(preprocessed_df)

Original DataFrame:
            job_id                                        description
73989   3902944011  The Senior Automation / Power Systems Engineer...
59308   3901960222  Company Summary\n\nDISH, an EchoStar Company, ...
44663   3900944095  Division: North Alabama\n\nDepartment : Oxford...
81954   3903878594  KMGH, the E.W. Scripps Company ABC affiliate i...
113151  3905670593  Come for the Flexibility, Stay for the Culture...
...            ...                                                ...
23623   3889770820  Environmental Health and Sustainability (EHS) ...
378     3805103054  Company Description\n An international non-pro...
4293    3884906708  What You'll Do\n\nAs an Internal Wholesaler – ...
93950   3904721422  Description\n\nAbout Us:\n\nUmpqua Bank is hea...
109452  3905361570  Job Summary\nThe Front-End Software Engineer I...

[1000 rows x 2 columns]

------------------------------

Preprocessing job descriptions...
Preprocessing complete!
Preprocessed DataFrame:


In [ ]:
def extract_skills(text: str) -> list:
    """
    Extracts entities from text using a pre-trained spaCy NER model.

    Args:
        text: The raw or preprocessed job description text.

    Returns:
        A list of extracted strings that are identified as potential skills.
    """
    if not isinstance(text, str):
        return []

    # Process the text with the spaCy NLP pipeline
    doc = nlp(text)

    # Simple filtering for potential skills.
    # This is a heuristic approach, as 'en_core_web_sm' doesn't have a 'SKILL' label.
    # We will primarily look for common nouns or noun phrases that might represent skills.
    
    # You can also iterate through `doc.ents` for a more explicit list of entities
    # recognized by the default model (e.g., ORG, GPE, DATE).
    
    # For a more robust approach, you'll need to create a list of skills to match
    # or fine-tune a model as discussed previously.
    
    # A simple example: extracting proper nouns and compound nouns
    skills = []
    for chunk in doc.noun_chunks:
        # Filter for phrases that are likely to be skills
        # This is a very basic filter; it will need to be refined.
        if "data" in chunk.text.lower() or "learning" in chunk.text.lower(): # TODO: Expand this list - make filter more comprehensive
            skills.append(chunk.text.strip())

    return list(set(skills)) # Return unique skills

In [42]:
def process_dataframe_for_skills(df: pd.DataFrame, text_column: str) -> pd.DataFrame:
    """
    Applies skill extraction to a DataFrame column.

    Args:
        df: The pandas DataFrame.
        text_column: The name of the column containing the job description text.

    Returns:
        A DataFrame with a new column 'extracted_skills'.
    """
    print("Starting skill extraction...")
    df['extracted_skills'] = df[text_column].apply(extract_skills)
    print("Skill extraction complete!")
    return df

In [43]:
# Apply the skill extraction function.
extracted_skills_df = process_dataframe_for_skills(preprocessed_df, 'description')

print("DataFrame with Extracted Skills:")
print(extracted_skills_df)

Starting skill extraction...
Skill extraction complete!
DataFrame with Extracted Skills:
            job_id                                        description  \
73989   3902944011  The Senior Automation / Power Systems Engineer...   
59308   3901960222  Company Summary\n\nDISH, an EchoStar Company, ...   
44663   3900944095  Division: North Alabama\n\nDepartment : Oxford...   
81954   3903878594  KMGH, the E.W. Scripps Company ABC affiliate i...   
113151  3905670593  Come for the Flexibility, Stay for the Culture...   
...            ...                                                ...   
23623   3889770820  Environmental Health and Sustainability (EHS) ...   
378     3805103054  Company Description\n An international non-pro...   
4293    3884906708  What You'll Do\n\nAs an Internal Wholesaler – ...   
93950   3904721422  Description\n\nAbout Us:\n\nUmpqua Bank is hea...   
109452  3905361570  Job Summary\nThe Front-End Software Engineer I...   

                                  

In [45]:
preprocessed_df

,job_id,description,preprocessed_text,extracted_skills
73989,3902944011,The Senior Automation / Power Systems Engineer...,Senior Automation Power Systems Engineer prima...,[]
59308,3901960222,"Company Summary\n\nDISH, an EchoStar Company, ...",Company Summary DISH EchoStar Company reimagin...,[]
44663,3900944095,Division: North Alabama\n\nDepartment : Oxford...,Division North Alabama Department Oxford Wareh...,[learning]
81954,3903878594,"KMGH, the E.W. Scripps Company ABC affiliate i...",KMGH EW Scripps Company ABC affiliate Denver C...,[]
113151,3905670593,"Come for the Flexibility, Stay for the Culture...",come Flexibility Stay Culture need life workli...,[]
...,...,...,...,...
23623,3889770820,Environmental Health and Sustainability (EHS) ...,Environmental Health Sustainability EHS Manage...,[technical data]
378,3805103054,Company Description\n An international non-pro...,Company Description international nonprofit or...,[]
4293,3884906708,What You'll Do\n\nAs an Internal Wholesaler – ...,ll Internal Wholesaler Small Market Retirement...,[]
93950,3904721422,Description\n\nAbout Us:\n\nUmpqua Bank is hea...,description Umpqua Bank headquarter Pacific No...,[]


In [47]:
extracted_skills_df

,job_id,description,preprocessed_text,extracted_skills
73989,3902944011,The Senior Automation / Power Systems Engineer...,Senior Automation Power Systems Engineer prima...,[]
59308,3901960222,"Company Summary\n\nDISH, an EchoStar Company, ...",Company Summary DISH EchoStar Company reimagin...,[]
44663,3900944095,Division: North Alabama\n\nDepartment : Oxford...,Division North Alabama Department Oxford Wareh...,[learning]
81954,3903878594,"KMGH, the E.W. Scripps Company ABC affiliate i...",KMGH EW Scripps Company ABC affiliate Denver C...,[]
113151,3905670593,"Come for the Flexibility, Stay for the Culture...",come Flexibility Stay Culture need life workli...,[]
...,...,...,...,...
23623,3889770820,Environmental Health and Sustainability (EHS) ...,Environmental Health Sustainability EHS Manage...,[technical data]
378,3805103054,Company Description\n An international non-pro...,Company Description international nonprofit or...,[]
4293,3884906708,What You'll Do\n\nAs an Internal Wholesaler – ...,ll Internal Wholesaler Small Market Retirement...,[]
93950,3904721422,Description\n\nAbout Us:\n\nUmpqua Bank is hea...,description Umpqua Bank headquarter Pacific No...,[]


## Trial and errors

In [11]:
#loading the spacy model (could give explanation why i chose en_core_web_sm model)
nlp = spacy.load("en_core_web_sm")

In [12]:
# removing the unnecessary columns
columns = ['job_id', 'title', 'description']
data1 = data[columns]
data1.shape

(123849, 3)

In [13]:
data1['title'].value_counts()

title
Sales Manager                                       673
Customer Service Representative                     373
Project Manager                                     354
Administrative Assistant                            254
Senior Accountant                                   238
                                                   ... 
Cath Lab / IR Technologist (Cert) - Cardiac Cath      1
Energy Administrative Assistant Part Time             1
ASSOCIATE CLIENT SUCCESS MANAGER                      1
Student Nurse - Telemetry                             1
Marketing Social Media Specialist                     1
Name: count, Length: 72521, dtype: int64

In [14]:
data1.head()

,job_id,title,description
0,921716,Marketing Coordinator,Job descriptionA leading real estate firm in N...
1,1829192,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ..."
2,10998357,Assitant Restaurant Manager,The National Exemplar is accepting application...
3,23221523,Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...
4,35982263,Service Technician,Looking for HVAC service tech with experience ...


In [15]:
# Sampling 5 data points
job_descriptions = data['description'].dropna().sample(5).tolist()
# Preprocessing function
def preprocess(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation and numbers
    doc = nlp(text)
    # tokenization and lemmatization (ex. running -> run)
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop and token.pos_ in {"NOUN", "PROPN"}]
    return tokens

In [16]:
# Apply preprocessing
processed_descriptions = [preprocess(desc) for desc in job_descriptions]

# Show results
for i, (raw, processed) in enumerate(zip(job_descriptions, processed_descriptions)):
    print(f"--- Job Description {i+1} ---")
    print("Original:", raw)
    print("------------------------------------------------------------------")
    print("Processed:", processed)
    print()

--- Job Description 1 ---
Original: Join an amazing team that is consistently recognized for our achievements and culture, including our most recent Forbes award of being one of America's Best Midsize Employers for 2024!

Position Summary

If you’re passionate about helping people restore their lives when the unexpected happens to their homes and providing the best customer experience, then our Mercury Insurance Property Claims team could be the place for you!

Upon completion of the training program, ideal candidates will transition into a property claims field adjusting position traveling to loss sites that have been damaged by fire, water, weather, or other unexpected events. You may also handle some claims via virtual technology and/or collaborate with vendors.

The Property Claims Field Adjuster ll will learn apply knowledge of current Company policies, applicable regulatory standards, and procedures to investigate, evaluate and settle moderate Homeowner's property claims in a tim

In [17]:
import re
from spacy import displacy

# Sample 5 job descriptions and drop NaN values
job_descriptions = data['description'].dropna().sample(5).tolist()

# Join the list into a single string
text = " ".join(job_descriptions).lower()  # Lowercase the combined text

# Remove punctuation and numbers
text = re.sub(r'[^a-zA-Z\s]', '', text)

# Process the cleaned text with spaCy
doc = nlp(text)

# Display named entities (if any)
displacy.render(doc, style="ent")


In [18]:
processed_descriptions = [preprocess(desc) for desc in job_descriptions]

In [19]:
entities = [(ent.text, ent.label_, ent.lemma_) for ent in doc.ents]
df = pd.DataFrame(entities, columns=['text', 'type', 'lemma'])
print(df)

                      text     type                  lemma
0                    third  ORDINAL                  third
1                one years     DATE               one year
2             twelve hours     TIME            twelve hour
3                 annually     DATE               annually
4                   weekly     DATE                 weekly
5                  monthly     DATE                monthly
6                    years     DATE                   year
7            massachusetts      GPE          massachusetts
8                   annual     DATE                 annual
9     ten to fifteen years     DATE    ten to fifteen year
10                     iep      ORG                    iep
11                one year     DATE               one year
12  at least  years of age     DATE  at least  year of age
13                  weekly     DATE                 weekly
14        maxim healthcare      ORG       maxim healthcare
15               a century     DATE              a centu

### NEXT STEPS

1. Skill Extraction - NER model for skills
Use a model trained to recognize skills (e.g., SpaCy custom NER, or libraries like SkillNer, Pyresparser, or ESCO-based tools).

2. Associate Skills with Job Titles

example:
skills_by_job = {}

for title, description in zip(job_titles, job_descriptions):
    doc = nlp(description.lower())
    tokens = [token.lemma_ for token in doc if token.lemma_ in skill_set]
    skills_by_job[title] = tokens


3. Clustering Skills into Categories - Automated clustering with embeddings (advanced)
Use word embeddings (like word2vec, spaCy, or sentence-transformers) + KMeans to group similar skills.

example:

from sklearn.cluster import KMeans
import numpy as np

--- Get unique skills
unique_skills = list(set(extracted_skills))

--- Get spaCy vector for each skill
skill_vectors = [nlp(skill).vector for skill in unique_skills]

--- Cluster with KMeans
kmeans = KMeans(n_clusters=5, random_state=42)
labels = kmeans.fit_predict(skill_vectors)

--- Map each skill to its cluster
clusters = {}
for skill, label in zip(unique_skills, labels):
    clusters.setdefault(label, []).append(skill)
